# SQL Conversations: Building an Intelligent Database Query Assistant

#### This notebook demonstrates how to create an AI-powered SQL agent that translates natural language questions into database queries, providing both accurate results and transparency into its reasoning process. By leveraging LangChain's SQL agent framework paired with a language model, we've built a system that allows non-technical users to extract insights from databases through simple questions while giving developers visibility into the agent's step-by-step thought process—from question interpretation to SQL formulation to answer generation.

#### It's fairly straightforward.  Step 1: connect to your llm and database.  Step 2: initialize your toolkit and agent with the llm and the db schema.  Step 3: run agent.  In roughly ~ 20 lines of code.  


### 1. Import libraries and dependencies

In [547]:
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.utilities import SQLDatabase
from langchain_community.agent_toolkits import SQLDatabaseToolkit
from langchain.agents import create_sql_agent
from langchain.agents import AgentExecutor
from langchain.prompts.prompt import PromptTemplate

import sqlite3
import json
import os

### 2. Setup LLM and API key

For our purposes, I'm using Gemini-1.5-pro.  But feel free to use others.

In [525]:

GOOGLE_API_KEY=  <YOUR_API_KEY>

llm = ChatGoogleGenerativeAI(model= "gemini-1.5-pro" , #"gemini-pro",
                           google_api_key=os.environ.get('GOOGLE_API_KEY'),
                           temperature=0.1)

results = llm.invoke("what are AI agents")

In [578]:
# Testing the connection
print(results.content)

AI agents are software programs that can act autonomously to achieve a given goal.  They perceive their environment, make decisions based on that perception, and then take actions to influence the environment.  Think of them as digital robots that operate in software environments rather than the physical world.

Here's a breakdown of key characteristics:

* **Autonomy:**  AI agents operate independently, making decisions and taking actions without constant human intervention.  They can adapt to changing circumstances and learn from experience.
* **Goal-Oriented:** They are designed with a specific objective in mind, whether it's playing a game, managing a schedule, or analyzing data.  Their actions are directed towards achieving this goal.
* **Perception:** They gather information about their environment through sensors.  In a virtual world, this might involve accessing data, reading text, or interpreting images.
* **Action:** They can perform actions that affect their environment.  Th

### 3. Connect to the (made-up) database and Do basic EDA

For this exercise I mocked up simple SQlite database, the schema of which is printed below.  

In [551]:
# Connect to the database
db_path = "sqlite:///ecommerce_sample.db"
db = SQLDatabase.from_uri(db_path)

In [557]:
# View schema and sample data
print(db.table_info)


CREATE TABLE categories (
	category_id INTEGER, 
	name TEXT NOT NULL, 
	description TEXT, 
	PRIMARY KEY (category_id)
)

/*
3 rows from categories table:
category_id	name	description
1	Electronics	Electronic gadgets and devices
2	Clothing	Apparel and accessories
3	Books	Books and e-books
*/


CREATE TABLE customers (
	customer_id INTEGER, 
	first_name TEXT NOT NULL, 
	last_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	phone TEXT, 
	address TEXT, 
	city TEXT, 
	state TEXT, 
	zip_code TEXT, 
	registration_date DATE DEFAULT CURRENT_DATE, 
	PRIMARY KEY (customer_id), 
	UNIQUE (email)
)

/*
3 rows from customers table:
customer_id	first_name	last_name	email	phone	address	city	state	zip_code	registration_date
1	John	Doe	john.doe@example.com	555-123-4567	123 Main St	Austin	TX	78701	2025-04-15
2	Jane	Smith	jane.smith@example.com	555-987-6543	456 Oak Ave	San Francisco	CA	94103	2025-04-15
3	Michael	Johnson	michael.j@example.com	555-234-5678	789 Pine Rd	New York	NY	10001	2025-04-15
*/


CREATE TA

#### 4. Test using a prompt

I decided to use Langchain's create_sql_agent(), a conversational AI agent that can interact with SQL databases. The toolkit provides the agent with database access capabilities, while the LangChain agent function connects this toolkit to a large language model (LLM). When executed, this agent interprets natural language queries (like "Who are the customers that have spent more than $500?"), translates them into SQL queries, executes those queries against the connected database, and returns the results in a human-readable format. The verbose flag enables detailed logging of the agent's thought process.

Just a general comment on the agent, it's interesting to see the 'thought' process under the hood.  I think it ultimately helps the developer better understand the logic and can also ultimately help in the debug process.  

In [559]:
# Create a toolkit and agent
toolkit = SQLDatabaseToolkit(db=db, llm=llm)
agent = create_sql_agent(
    llm=llm,
    toolkit=toolkit,
    verbose=True
)

# Run the agent
result = agent.run("Who are the customers that have spent more than $500?")
print(result)



> Entering new SQL Agent Executor chain...
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: categories, customers, order_items, orders, productsThought: The `customers` and `orders` tables seem relevant. I'll check their schema.
Action: sql_db_schema
Action Input: customers, orders
CREATE TABLE customers (
	customer_id INTEGER, 
	first_name TEXT NOT NULL, 
	last_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	phone TEXT, 
	address TEXT, 
	city TEXT, 
	state TEXT, 
	zip_code TEXT, 
	registration_date DATE DEFAULT CURRENT_DATE, 
	PRIMARY KEY (customer_id), 
	UNIQUE (email)
)

/*
3 rows from customers table:
customer_id	first_name	last_name	email	phone	address	city	state	zip_code	registration_date
1	John	Doe	john.doe@example.com	555-123-4567	123 Main St	Austin	TX	78701	2025-04-15
2	Jane	Smith	jane.smith@example.com	555-987-6543	456 Oak Ave	San Francisco	CA	9410

#### 4. Testing with more complex questions

In this example, I want to see if the agent can figure out that it needs 

In [600]:

result = agent.run("who are top 3 customers and what did they buy?")
print(result)



> Entering new SQL Agent Executor chain...
Thought: I should look at the tables in the database to see what I can query.  Then I should query the schema of the most relevant tables.
Action: sql_db_list_tables
Action Input: categories, customers, order_items, orders, productsThought: The relevant tables are customers, orders, order_items, and products. I'll check their schema.
Action: sql_db_schema
Action Input: customers, orders, order_items, products
CREATE TABLE customers (
	customer_id INTEGER, 
	first_name TEXT NOT NULL, 
	last_name TEXT NOT NULL, 
	email TEXT NOT NULL, 
	phone TEXT, 
	address TEXT, 
	city TEXT, 
	state TEXT, 
	zip_code TEXT, 
	registration_date DATE DEFAULT CURRENT_DATE, 
	PRIMARY KEY (customer_id), 
	UNIQUE (email)
)

/*
3 rows from customers table:
customer_id	first_name	last_name	email	phone	address	city	state	zip_code	registration_date
1	John	Doe	john.doe@example.com	555-123-4567	123 Main St	Austin	TX	78701	2025-04-15
2	Jane	Smith	jane.smith@example.com	555-

In [604]:
print(result)

The top 3 customers are: 1. John Doe spent $2039.9 and bought Smartphone X, Wireless Headphones, T-shirt Basic, and Yoga Mat. 2. Michael Johnson spent $1299.99 and bought Laptop Pro. 3. Jane Smith spent $1049.88 and bought T-shirt Basic, SQL Database Design, Coffee Maker, Jeans Classic, and Basketball.


#### 5. Future enhancements

The internal prompt is setup to output the final answer in string format.  A future enhancement might be output in tabular format.